# Descripción Estadística de Casos Nuevos diarios, Fallecimientos acumulados y Ocupación de Camas UCI diarias por COVID para V Region
por Dr. Egidio Céspedes, Científico de Datos, SEREMI V Región

Datos          : https://github.com/MinCiencia/Datos-COVID19

Fuente Casos Chile   : Ministerio de Salud. Ver en: https://www.minsal.cl/nuevo-coronavirus-2019-ncov/casos-confirmados-en-chile-covid-19/

Fuente Casos Mundo   :  Our World in Data. Oxford Martin School, University of Oxford. Ver en: https://ourworldindata.org/coronavirus-source-data 

Fuente UCI Chile    : Reporte diario del Ministerio de Salud. Ver en: https://www.gob.cl/coronavirus/cifrasoficiales/#reportes

Fuente Muertos : Reporte diario del Ministerio de Salud. Ver en: https://www.gob.cl/coronavirus/cifrasoficiales/#reportes

Fuente Muertos Mundo   :  Our World in Data. Oxford Martin School, University of Oxford. Ver en: https://ourworldindata.org/coronavirus-source-data

Datos de Actualización Diaria

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Se leen las BDs desde GitHub directamente

In [2]:
#Se leen las BDs chilenas desde GitHub directamente.
TxR = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/TotalesPorRegion_T.csv")
UCI = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto8/UCI_T.csv")
MxD = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo_T.csv")

#Se leen las BDs del mundo desde GitHub directamente
NewCases=pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/new_cases.csv")
Deads=pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/new_deaths.csv")

Se modifican las BDs, se dejan solo datos necesarios y se dejan columnas en formato fecha correspondiente

In [3]:
def muertos(df):
    a=0
    lista=[]
    for i in df.Valparaíso:
        lista.append(i-a)
        a=i
    df['Valparaíso']=lista
    
    b=0
    lista2=[]
    for i in df.Total:
        lista2.append(i-b)
        b=i
    df['Total']=lista2

In [4]:
Casos=pd.concat([TxR[['Region']].drop(TxR[['Region']].index[[0]]),
                 TxR.iloc[:,19:35].drop(TxR.iloc[:,19:34].index[[0]]).apply(pd.to_numeric).apply(pd.to_numeric)], 
                axis=1)
Casos=Casos.rename(columns={'Region':'Fecha','Valparaíso.1':'Valparaíso','Total.1':'Total'})
Casos.Fecha=pd.to_datetime(Casos.Fecha)

MxD=MxD.rename(columns={'Region':'Fecha'})
MxD.Fecha=pd.to_datetime(MxD.Fecha)
muertos(MxD)

UCI=UCI.rename(columns={'Region':'Fecha'})
UCI=pd.concat([UCI.Fecha.drop(UCI.Fecha.index[[0,1]]),
               UCI.iloc[:,1:].drop(UCI.index[[0,1]]).apply(pd.to_numeric)],
              axis=1)
drop=UCI.drop(['Fecha'], axis=1)
UCI.Fecha=pd.to_datetime(UCI.Fecha)
UCI['Total']=drop.sum(axis=1)

Se crean las funciones que generan el reporte

In [5]:
def informe_nac(df,name):
    M=pd.DataFrame([df['Total'].mean(),
                    df['Total'].sum(),
                    df['Total'].mean().pct_change()])
    M=M.T
    M.columns = ['Prom_Sem_Nac'+name,'Total_Nac'+name,'Velocidad_Nac'+name]
    M.drop(M.tail(1).index,inplace=True)
    return M

def informe_valpo(df,name):
    M=pd.DataFrame([df['Valparaíso'].mean(),
                    df['Valparaíso'].sum(),
                    df['Valparaíso'].mean().pct_change()])
    M=M.T
    M.columns = ['Prom_Sem_Valp'+name,'Total_Valp'+name,'Velocidad_Valp'+name]
    M.drop(M.tail(1).index,inplace=True)
    return M

def informe_pais(df1,df2,name):
    df1.rename(columns={'date': 'Fecha'}, inplace=True)
    df1.Fecha=pd.to_datetime(NewCases.Fecha)
    agrupados1=df1.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
    M=pd.DataFrame([agrupados1[name].mean(),
                    agrupados1[name].sum(),
                    agrupados1[name].mean().pct_change()])
    M=M.T
    M.columns = ['Prom_Sem_Casos_'+name,'Total_Sem_Casos_'+name,'Velocidad_Sem_Casos_'+name]
    
    df2.rename(columns={'date': 'Fecha'}, inplace=True)
    df2.Fecha=pd.to_datetime(NewCases.Fecha)
    agrupados2=df2.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
    F=pd.DataFrame([agrupados2[name].mean(),
                    agrupados2[name].sum(),
                    agrupados2[name].mean().pct_change()])
    F=F.T
    F.columns = ['Prom_Sem_Muertes_'+name,'Total_Sem_Muertes_'+name,'Velocidad_Muertes_Casos_'+name]
    
    Z=pd.concat([M,F],axis=1)
    #Z.drop(Z.tail(1).index,inplace=True)
    Z=Z.tail(10)
    return Z

In [6]:
Casos2 = Casos.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
UCI2   = UCI.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
MxD2   = MxD.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])

In [14]:
Valpo =pd.concat([informe_valpo(Casos2,'_Casos').tail(10),
           informe_valpo(UCI2,'_UCI').tail(10),
           informe_valpo(MxD2,'_Muertes').tail(10)], 
                 axis=1)

In [8]:
Chile=pd.concat([informe_nac(Casos2,'_Casos').tail(10),
           informe_nac(UCI2,'_UCI').tail(10),
           informe_nac(MxD2,'_Muertes').tail(10)],
                axis=1)

In [41]:
list_semana = ['Semana -10', 'Semana -9', 'Semana -8', 'Semana -7', 'Semana -6', 'Semana -5', 'Semana -4', 'Semana -3', 'Semana -2', 'Semana -1' ]
Valpo['Semana comparacion'] = list_semana
Valpo = Valpo[['Semana comparacion', 'Prom_Sem_Valp_Casos', 'Total_Valp_Casos', 'Velocidad_Valp_Casos',
       'Prom_Sem_Valp_UCI', 'Total_Valp_UCI', 'Velocidad_Valp_UCI',
       'Prom_Sem_Valp_Muertes', 'Total_Valp_Muertes', 'Velocidad_Valp_Muertes',
       ]]

In [48]:
Valpo.columns = [
    'Semana comparacion', 
    'Prom. Diario', 'Total Semana', 'Velocidad Semana',
    'Prom. Diario', 'Total Semana', 'Velocidad Semana',
    'Prom. Diario', 'Total Semana', 'Velocidad Semana']
Valpo.index.rename('Fecha semana', inplace=True)

In [53]:
Valpo.to_latex()

'\\begin{tabular}{llrrrrrrrrr}\n\\toprule\n{} & Semana comparacion &  Prom. Diario &  Total Semana &  Velocidad Semana &  Prom. Diario &  Total Semana &  Velocidad Semana &  Prom. Diario &  Total Semana &  Velocidad Semana \\\\\nFecha semana &                    &               &               &                   &               &               &                   &               &               &                   \\\\\n\\midrule\n2022-05-15   &         Semana -10 &    414.142857 &        2899.0 &          0.388410 &     15.285714 &         107.0 &         -0.176923 &      2.000000 &          14.0 &         -0.363636 \\\\\n2022-05-22   &          Semana -9 &    626.714286 &        4387.0 &          0.513280 &     13.000000 &          91.0 &         -0.149533 &      1.285714 &           9.0 &         -0.357143 \\\\\n2022-05-29   &          Semana -8 &    870.285714 &        6092.0 &          0.388648 &      9.428571 &          66.0 &         -0.274725 &      1.571429 &          11.0 & 

In [59]:
print(Valpo.to_latex(float_format="%0.1f"))

\begin{tabular}{llrrrrrrrrr}
\toprule
{} & Semana comparacion &  Prom. Diario &  Total Semana &  Velocidad Semana &  Prom. Diario &  Total Semana &  Velocidad Semana &  Prom. Diario &  Total Semana &  Velocidad Semana \\
Fecha semana &                    &               &               &                   &               &               &                   &               &               &                   \\
\midrule
2022-05-15   &         Semana -10 &         414.1 &        2899.0 &               0.4 &          15.3 &         107.0 &              -0.2 &           2.0 &          14.0 &              -0.4 \\
2022-05-22   &          Semana -9 &         626.7 &        4387.0 &               0.5 &          13.0 &          91.0 &              -0.1 &           1.3 &           9.0 &              -0.4 \\
2022-05-29   &          Semana -8 &         870.3 &        6092.0 &               0.4 &           9.4 &          66.0 &              -0.3 &           1.6 &          11.0 &               0.2 \

In [10]:
Chile

,Prom_Sem_Nac_Casos,Total_Nac_Casos,Velocidad_Nac_Casos,Prom_Sem_Nac_UCI,Total_Nac_UCI,Velocidad_Nac_UCI,Prom_Sem_Nac_Muertes,Total_Nac_Muertes,Velocidad_Nac_Muertes
Fecha,,,,,,,,,
2022-05-15,3353.285714,23473.0,0.419938,169.000000,1183.0,-0.117164,12.142857,85.0,-0.132653
2022-05-22,4814.714286,33703.0,0.435820,148.571429,1040.0,-0.120879,11.428571,80.0,-0.058824
2022-05-29,6219.285714,43535.0,0.291725,130.714286,915.0,-0.120192,10.714286,75.0,-0.062500
2022-06-05,6861.571429,48031.0,0.103273,145.714286,1020.0,0.114754,13.714286,96.0,0.280000
2022-06-12,8923.571429,62465.0,0.300514,159.285714,1115.0,0.093137,17.428571,122.0,0.270833
2022-06-19,9631.571429,67421.0,0.079340,163.857143,1147.0,0.028700,22.285714,156.0,0.278689
2022-06-26,8777.857143,61445.0,-0.088637,185.857143,1301.0,0.134263,23.571429,165.0,0.057692
2022-07-03,8679.571429,60757.0,-0.011197,179.000000,1253.0,-0.036895,24.714286,173.0,0.048485
2022-07-10,8212.857143,57490.0,-0.053772,184.571429,1292.0,0.031125,32.714286,229.0,0.323699


In [11]:
informe_pais(NewCases,Deads,'Argentina')

,Prom_Sem_Casos_Argentina,Total_Sem_Casos_Argentina,Velocidad_Sem_Casos_Argentina,Prom_Sem_Muertes_Argentina,Total_Sem_Muertes_Argentina,Velocidad_Muertes_Casos_Argentina
Fecha,,,,,,
2022-05-22,6212.428571,43487.0,0.279443,7.000000,49.0,0.042553
2022-05-29,7396.857143,51778.0,0.190655,9.142857,64.0,0.306122
2022-06-05,6577.857143,46045.0,-0.110723,12.000000,84.0,0.312500
2022-06-12,0.000000,0.0,-1.000000,0.000000,0.0,-1.000000
2022-06-19,9267.714286,64874.0,inf,6.142857,43.0,inf
2022-06-26,3668.571429,25680.0,-0.604156,7.714286,54.0,0.255814
2022-07-03,3879.142857,27154.0,0.057399,5.571429,39.0,-0.277778
2022-07-10,4549.285714,31845.0,0.172755,5.142857,36.0,-0.076923
2022-07-17,5665.142857,39656.0,0.245282,8.142857,57.0,0.583333


In [12]:
informe_pais(NewCases,Deads,'Peru')

,Prom_Sem_Casos_Peru,Total_Sem_Casos_Peru,Velocidad_Sem_Casos_Peru,Prom_Sem_Muertes_Peru,Total_Sem_Muertes_Peru,Velocidad_Muertes_Casos_Peru
Fecha,,,,,,
2022-05-22,1216.400000,6082.0,2.224082,15.500000,93.0,0.154255
2022-05-29,572.714286,4009.0,-0.529173,9.571429,67.0,-0.382488
2022-06-05,761.428571,5330.0,0.329509,12.285714,86.0,0.283582
2022-06-12,785.857143,5501.0,0.032083,9.285714,65.0,-0.244186
2022-06-19,1231.285714,8619.0,0.566806,9.285714,65.0,0.000000
2022-06-26,2489.714286,17428.0,1.022044,10.428571,73.0,0.123077
2022-07-03,2942.857143,20600.0,0.182006,12.714286,89.0,0.219178
2022-07-10,5374.714286,37623.0,0.826359,19.142857,134.0,0.505618
2022-07-17,9387.000000,65709.0,0.746511,16.571429,116.0,-0.134328


In [13]:
Chile.to_latex()

'\\begin{tabular}{lrrrrrrrrr}\n\\toprule\n{} &  Prom\\_Sem\\_Nac\\_Casos &  Total\\_Nac\\_Casos &  Velocidad\\_Nac\\_Casos &  Prom\\_Sem\\_Nac\\_UCI &  Total\\_Nac\\_UCI &  Velocidad\\_Nac\\_UCI &  Prom\\_Sem\\_Nac\\_Muertes &  Total\\_Nac\\_Muertes &  Velocidad\\_Nac\\_Muertes \\\\\nFecha      &                     &                  &                      &                   &                &                    &                       &                    &                        \\\\\n\\midrule\n2022-05-15 &         3353.285714 &          23473.0 &             0.419938 &        169.000000 &         1183.0 &          -0.117164 &             12.142857 &               85.0 &              -0.132653 \\\\\n2022-05-22 &         4814.714286 &          33703.0 &             0.435820 &        148.571429 &         1040.0 &          -0.120879 &             11.428571 &               80.0 &              -0.058824 \\\\\n2022-05-29 &         6219.285714 &          43535.0 &             0.291725 & 